<a href="https://colab.research.google.com/github/abdurrahmanshidiq/IBM-Capstone-Project/blob/master/Segmenting_and_Clustering_Neighborhoods_in_Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in New York City</font></h1>

# Table of Content


1. Chapter 1 : Scrapping Wikipedia to get Toronto Neighborhoods Data 
2. Chapter 2 : Load Geolocation Data, then Merged to Toronto Neighborhoods Data based on Postal Code
3. Chapter 3 : Explore & Cluster the Neighborhoods in Toronto using Foursquare API & K-Means Clustering

# Chapter 1

Scrapping Wikipedia to get *Toronto Neighborhoods Data*

In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

from bs4 import BeautifulSoup
import requests
import json
from geopy.geocoders import Nominatim
from pandas import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

import folium
from folium import plugins

from sklearn.cluster import KMeans

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw = requests.get(url).text

In [3]:
soup = BeautifulSoup(raw, 'html5lib')

In [4]:
len(soup.find('table'))

2

In [5]:
table = soup.find_all('table')[0].find_all('td')

post_code = []
borough = []
neigh = []

for i,row in enumerate(table):
  if table[i].span.text == 'Not assigned':
    pass
  else:
    post_code.append(table[i].p.text[:3])
    borough.append(table[i].span.text.split('(')[0])
    neigh.append(table[i].span.text.split('(')[1].replace('/',',').replace(')',''.strip(' ')))

**Click Here for alternative code**
<!--
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
-->

In [6]:
#Checking Unique Values for Borough

df_toronto = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])
df_toronto['PostalCode'] = post_code
df_toronto['Borough'] = borough
df_toronto['Neighborhood'] = neigh

df_toronto['Borough'].unique()

array(['North York', 'Downtown Toronto', "Queen's Park", 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'East YorkEast Toronto', 'Central Toronto',
       'MississaugaCanada Post Gateway Processing Centre',
       'Downtown TorontoStn A PO Boxes25 The Esplanade',
       'EtobicokeNorthwest',
       'East TorontoBusiness reply mail Processing Centre969 Eastern'],
      dtype=object)

In [7]:
# Fixing Borough Name
# Getting the Data Index
print(df_toronto[df_toronto['Borough']=='East YorkEast Toronto'].index)
print(df_toronto[df_toronto['Borough']=='MississaugaCanada Post Gateway Processing Centre'].index)
print(df_toronto[df_toronto['Borough']=='Downtown TorontoStn A PO Boxes25 The Esplanade'].index)
print(df_toronto[df_toronto['Borough']=='EtobicokeNorthwest'].index)
print(df_toronto[df_toronto['Borough']=='East TorontoBusiness reply mail Processing Centre969 Eastern'].index)

Int64Index([35], dtype='int64')
Int64Index([76], dtype='int64')
Int64Index([92], dtype='int64')
Int64Index([94], dtype='int64')
Int64Index([100], dtype='int64')


In [8]:
# Fixing Borough Name
# Replace wrong `Borough` name with the correct name
df_toronto.loc[35,'Borough'] = df_toronto.loc[35,'Borough'].replace('East YorkEast Toronto', 'East York')
df_toronto.loc[76,'Borough'] = df_toronto.loc[76,'Borough'].replace('MississaugaCanada Post Gateway Processing Centre', 'Mississauga')
df_toronto.loc[92,'Borough'] = df_toronto.loc[92,'Borough'].replace('Downtown TorontoStn A PO Boxes25 The Esplanade', 'Downtown Toronto')
df_toronto.loc[94,'Borough'] = df_toronto.loc[94,'Borough'].replace('EtobicokeNorthwest', 'Etobicoke')
df_toronto.loc[100,'Borough'] = df_toronto.loc[100,'Borough'].replace('East TorontoBusiness reply mail Processing Centre969 Eastern', 'East Toronto')

In [9]:
df_toronto.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [10]:
#Cheking for missing values
df_toronto.isnull().sum()

PostalCode      0
Borough         0
Neighborhood    0
dtype: int64

In [11]:
df_toronto.shape

(103, 3)

**End of Chapter 1**

# Chapter 2

Adding Latitude & Longitude to the Data Frame

In [12]:
# Import csv file to google colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
df_geo = pd.read_csv('/content/drive/My Drive/IBM DS Certificate/10. Capstone Project/2. K-Means Clustering/Geospatial_Coordinates.csv')
df_geo.columns = ['PostalCode', 'Latitude', 'Longitude']
df_geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
df_geo.shape

(103, 3)

In [15]:
# Merge `df_toronto` with `df_geo`based on PostalCode
df_merged = df_toronto.merge(df_geo, how='inner', on='PostalCode')
df_merged = df_merged[['Borough', 'Neighborhood', 'Latitude', 'Longitude']]

**End of Chapter 2**

# Chapter 3

Explore and Cluster the neighborhoods in Toronto

In [16]:
neighborhoods = df_merged.copy()
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [17]:
print('The dataframe has {} borough & {} neighborhoods.'.format(len(neighborhoods['Borough'].unique()), neighborhoods.shape[0]))

The dataframe has 11 borough & 103 neighborhoods.


## Visualizing Toronto's Borough

In [18]:
# Use Geopy to get Lat, Lon of Toronto

address = 'Toronto, ON'

geolocator = Nominatim(user_agent='on_explore')
location = geolocator.geocode(address)
lat = location.latitude
lon = location.longitude
print('Geographical Coordinate of Toronto are {}, {}'.format(lat, lon))

Geographical Coordinate of Toronto are 43.6534817, -79.3839347


In [19]:
# Create Map of Toronto
map_toronto = folium.Map(location=[lat,lon], zoom_start=10)

for lat, lon, borough, neigh in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
  label = 'Neighborhood: {} \n\n Borough: {}'.format(neigh, borough)
  label = folium.Popup(label, parse_html=True)
  folium.CircleMarker([lat,lon], radius=5,
                      popup=label, color='blue',
                      fill=True, fill_color='#3186cc',
                      fill_opacity=0.5, parse_html=False).add_to(map_toronto)

map_toronto

In [20]:
neighborhoods['Borough'].value_counts()

North York          24
Downtown Toronto    18
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East York            5
East Toronto         5
Queen's Park         1
Mississauga          1
Name: Borough, dtype: int64

Simplify the above map and segment and cluster only the neighborhoods in `North York`. So let's slice the original dataframe and create a new dataframe of the `North York` data.

## Lets Explore `North York` Borough

In [21]:
# Select Only North York Borough

df_NY = neighborhoods[neighborhoods['Borough']=='North York'].reset_index(drop=True)[['Borough', 'Neighborhood', 'Latitude', 'Longitude']]
df_NY.head()

,Borough,Neighborhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
3,North York,Don MillsNorth,43.745906,-79.352188
4,North York,Glencairn,43.709577,-79.445073


In [22]:
df_NY.shape

(24, 4)

In [23]:
# Get Coordiates for North York, Toronto, Canada
address = 'North York, ON, Canada'

geolocator = Nominatim(user_agent='on_explore')
location = geolocator.geocode(address)
lat = location.latitude
lon = location.longitude
print('Geographical Coordinate of North York, Toronto are {}, {}'.format(lat, lon))

Geographical Coordinate of North York, Toronto are 43.7543263, -79.44911696639593


In [24]:
# Visualizing North York Map

map_NorthYork = folium.Map(location=[lat,lon], zoom_start=11)

for lat, lon, neigh in zip(df_NY['Latitude'], df_NY['Longitude'], df_NY['Neighborhood']):
  label = '{}'.format(neigh)
  label = folium.Popup(label, parse_html=True)
  folium.CircleMarker([lat,lon], radius=5,
                      popup=label, color='blue',
                      fill=True, fill_color='#3186cc',
                      fill_opacity=0.5, parse_html=False).add_to(map_NorthYork)

map_NorthYork

In [25]:
map_NorthYork = folium.Map(location=[lat,lon], zoom_start=11)

incidents = plugins.MarkerCluster().add_to(map_NorthYork)

for lat, lon, neigh in zip(df_NY['Latitude'], df_NY['Longitude'], df_NY['Neighborhood']):
  label = '{}'.format(neigh)
  label = folium.Popup(label, parse_html=True)
  folium.CircleMarker([lat,lon], radius=5,
                      popup=label, color='blue',
                      fill=True, fill_color='#3186cc',
                      fill_opacity=0.5, parse_html=False).add_to(incidents)

map_NorthYork

## Utilizing the Foursquare API to explore the neighborhoods

Lets Explore `Parkwoods` Neighborhood

In [26]:
CLIENT_ID = '3UT2CP1TASOWFFEBJL3DGSHWTXORPEA5H1UQLX02M3HGHDS1' # your Foursquare ID
CLIENT_SECRET = 'P4N2G2HR4TKTXLDEEVUT1EWROXQ3ZSRHO1TFGGZ2JAWKEDIS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3UT2CP1TASOWFFEBJL3DGSHWTXORPEA5H1UQLX02M3HGHDS1
CLIENT_SECRET:P4N2G2HR4TKTXLDEEVUT1EWROXQ3ZSRHO1TFGGZ2JAWKEDIS


Parkwoods-North York-Toronto-Canada

In [27]:
df_NY.head()

,Borough,Neighborhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
3,North York,Don MillsNorth,43.745906,-79.352188
4,North York,Glencairn,43.709577,-79.445073


In [28]:
# Parkwoods

neigh_name = df_NY.loc[0,'Neighborhood']
neigh_lat = df_NY.loc[0,'Latitude']
neigh_lon = df_NY.loc[0,'Longitude']

print('Coordinate for "{}" neighborhood is = Lat : {}, Lng : {}'.format(neigh_name, neigh_lat, neigh_lon))

Coordinate for "Parkwoods" neighborhood is = Lat : 43.7532586, Lng : -79.3296565


End Point Venue/Explore = Returns a list of recommended venues near the current location.

 Get the top 100 venues that are in `Parkwoods` within a radius of 500 meters.

In [29]:
# Get the top 100 venues that are in Parkwoods within a radius of 500 meters.
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET,
                                                                                                                           VERSION, neigh_lat, neigh_lon,
                                                                                                                           radius, LIMIT)
print(url)

https://api.foursquare.com/v2/venues/explore?client_id=3UT2CP1TASOWFFEBJL3DGSHWTXORPEA5H1UQLX02M3HGHDS1&client_secret=P4N2G2HR4TKTXLDEEVUT1EWROXQ3ZSRHO1TFGGZ2JAWKEDIS&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100


In [30]:
results = requests.get(url).json()
len(results['response']['groups'][0]['items'])

3

### Inspecting Json File ##

In [31]:
res = requests.get(url).json()

In [32]:
res.keys()

dict_keys(['meta', 'response'])

In [33]:
res['response'].keys()

dict_keys(['warning', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'totalResults', 'suggestedBounds', 'groups'])

In [34]:
json_normalize(res['response']['groups'][0]['items']).head()

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.venuePage.id
0,e-0-4e8d9dcdd5fbbbb6b3003c7b-0,0,"[{'summary': 'This spot is popular', 'type': '...",4e8d9dcdd5fbbbb6b3003c7b,Brookbanks Park,Toronto,43.751976,-79.332140,"[{'label': 'display', 'lat': 43.75197604605557...",245,CA,Toronto,ON,Canada,"[Toronto, Toronto ON, Canada]","[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",0,[],600917367
1,e-0-4e6696b6d16433b9ffff47c3-1,0,"[{'summary': 'This spot is popular', 'type': '...",4e6696b6d16433b9ffff47c3,KFC,NaN,43.754387,-79.333021,"[{'label': 'display', 'lat': 43.75438666345904...",298,CA,NaN,NaN,Canada,[Canada],"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",0,[],NaN
2,e-0-4cb11e2075ebb60cd1c4caad-2,0,"[{'summary': 'This spot is popular', 'type': '...",4cb11e2075ebb60cd1c4caad,Variety Store,29 Valley Woods Road,43.751974,-79.333114,"[{'label': 'display', 'lat': 43.75197441585782...",312,CA,Toronto,ON,Canada,"[29 Valley Woods Road, Toronto ON, Canada]","[{'id': '4bf58dd8d48988d1f9941735', 'name': 'F...",0,[],NaN


### Extracting Nearby `Venues` Information from JSON file

- Create function to extract nearby venues for all neighborhoods

In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
  neigh_name = []
  neigh_lat = []
  neigh_lon = []
  venue_name = []
  venue_lat = []
  venue_lon = []
  venue_cat_name = []
  print('Neighborhoods in North York, Toronto : \n')
  for name, lat, lon in zip(names, latitudes, longitudes):
    print(name)

    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET,
                                                                                                                           VERSION, lat, lon,
                                                                                                                           radius, LIMIT)
    results = requests.get(url).json()['response']['groups'][0]['items']

    for val in results:
      if len(val)<1:
        pass
      else:
        neigh_name.append(name)
        neigh_lat.append(lat)
        neigh_lon.append(lon)
        venue_name.append(val['venue']['name'])
        venue_lat.append(val['venue']['location']['lat'])
        venue_lon.append(val['venue']['location']['lng'])
        venue_cat_name.append(val['venue']['categories'][0]['name'])


  nearby_venue = pd.DataFrame({'Neighborhood':neigh_name, 'Neighboorhood Latitude':neigh_lat, 'Neighborhood Longitude':neigh_lon, 
                               'Venue':venue_name, 'Venue Latitude':venue_lat, 'Venue Longitude':venue_lon, 'Venue Category':venue_cat_name})
  return nearby_venue

In [36]:
df_merged.head()

,Borough,Neighborhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [37]:
df_merged.shape

(103, 4)

In [38]:
toronto_venues = getNearbyVenues(names=df_merged['Neighborhood'],
                                   latitudes=df_merged['Latitude'],
                                   longitudes=df_merged['Longitude']
                                  )

Neighborhoods in North York, Toronto : 

Parkwoods
Victoria Village
Regent Park , Harbourfront
Lawrence Manor , Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern , Rouge
Don MillsNorth
Parkview Hill , Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park , Princess Gardens , Martin Grove , Islington , Cloverdale
Rouge Hill , Port Union , Highland Creek
Don MillsSouth
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate , Bloordale Gardens , Old Burnhamthorpe , Markland Wood
Guildwood , Morningside , West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor , Wilson Heights , Downsview North
Thorncliffe Park
Richmond , Adelaide , King
Dufferin , Dovercourt Village
Scarborough Village
Fairview , Henry Farm , Oriole
Northwood Park , York University
The Danforth  East
Harbourfront East , Union Station , Toronto Islands
Little Portugal , Trinity
Kennedy Park , Ionview

In [39]:
# All nearby venues in toronto
toronto_venues.head()

,Neighborhood,Neighboorhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [40]:
toronto_venues.shape

(2133, 7)

In [41]:
#Checking null values
toronto_venues.isnull().sum()

Neighborhood              0
Neighboorhood Latitude    0
Neighborhood Longitude    0
Venue                     0
Venue Latitude            0
Venue Longitude           0
Venue Category            0
dtype: int64

In [42]:
#Top 10 Neighborhood with number of venues
toronto_venues.groupby('Neighborhood').count().reset_index()[['Neighborhood','Venue']].sort_values('Venue', ascending=False).head(10)

,Neighborhood,Venue
34,"First Canadian Place , Underground city",100
17,"Commerce Court , Victoria Hotel",100
40,"Harbourfront East , Union Station , Toronto Is...",100
36,"Garden District, Ryerson",100
87,"Toronto Dominion Centre , Design Exchange",100
68,"Richmond , Adelaide , King",100
31,Enclave of M5E,99
76,St. James Town,86
13,Church and Wellesley,77
11,Central Bay Street,68


In [43]:
print('There are {} uniques venue categories in Toronto-Canada.'.format(len(toronto_venues['Venue Category'].unique())))

There are 276 uniques venue categories in Toronto-Canada.


### Analyze Each Neighborhood

In [44]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [45]:
toronto_onehot.shape

(2133, 276)

In [46]:
# let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.000000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000

In [47]:
toronto_grouped.shape

(101, 276)

In [48]:
# Let's print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt ----
                       venue  freq
0             Breakfast Spot  0.25
1                     Lounge  0.25
2  Latin American Restaurant  0.25
3         Chinese Restaurant  0.25
4                Yoga Studio  0.00


----Alderwood , Long Branch----
            venue  freq
0     Pizza Place  0.25
1        Pharmacy  0.12
2     Coffee Shop  0.12
3  Sandwich Place  0.12
4             Pub  0.12


----Bathurst Manor , Wilson Heights , Downsview North----
                 venue  freq
0                 Bank  0.08
1          Coffee Shop  0.08
2            Gift Shop  0.04
3    Mobile Phone Shop  0.04
4  Fried Chicken Joint  0.04


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4                Motel  0.00


----Bedford Park , Lawrence Manor East----
                     venue  freq
0              Coffee Shop  0.08
1           Sandwich Place  0.08
2       Italian

**Let's put that into a pandas dataframe**

In [49]:
# First, let's write a function to sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

**Now let's create the new dataframe and display the top 10 venues for each neighborhood.**

In [50]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Breakfast Spot,Chinese Restaurant,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
1,"Alderwood , Long Branch",Pizza Place,Coffee Shop,Gym,Pharmacy,Sandwich Place,Pub,Pool,Distribution Center,Dessert Shop,Dim Sum Restaurant
2,"Bathurst Manor , Wilson Heights , Downsview North",Coffee Shop,Bank,Park,Fried Chicken Joint,Deli / Bodega,Supermarket,Middle Eastern Restaurant,Sushi Restaurant,Ice Cream Shop,Shopping Mall
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Diner,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,"Bedford Park , Lawrence Manor East",Sandwich Place,Coffee Shop,Italian Restaurant,Comfort Food Restaurant,Café,Pub,Sushi Restaurant,Butcher,Liquor Store,Restaurant


## ***Cluster Neighborhoods*** in `Toronto` based on nearby venues

In [51]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       2, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 4, 2], dtype=int32)

In [52]:
# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_merged

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,43.753259,-79.329656,2.0,Fast Food Restaurant,Park,Food & Drink Shop,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dessert Shop
1,North York,Victoria Village,43.725882,-79.315572,0.0,Pizza Place,Coffee Shop,Hockey Arena,Portuguese Restaurant,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
2,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,0.0,Coffee Shop,Park,Pub,Bakery,Theater,Breakfast Spot,Café,Historic Site,Spa,Beer Store
3,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763,0.0,Clothing Store,Coffee Shop,Furniture / Home Store,Miscellaneous Shop,Carpet Store,Boutique,Vietnamese Restaurant,Accessories Store,Electronics Store,Eastern European Restaurant
4,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,0.0,Coffee Shop,Diner,Café,Sushi Restaurant,Yoga Studio,Portuguese Restaurant,Bar,Beer Bar,Spa,Smoothie Shop


In [53]:
#Checking NaN
toronto_merged.isnull().sum()

Borough                   0
Neighborhood              0
Latitude                  0
Longitude                 0
Cluster Labels            2
1st Most Common Venue     2
2nd Most Common Venue     2
3rd Most Common Venue     2
4th Most Common Venue     2
5th Most Common Venue     2
6th Most Common Venue     2
7th Most Common Venue     2
8th Most Common Venue     2
9th Most Common Venue     2
10th Most Common Venue    2
dtype: int64

In [54]:
#Checking NaN
toronto_merged[toronto_merged.isnull().any(axis=1)]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Etobicoke,Islington Avenue,43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,Scarborough,Upper Rouge,43.836125,-79.205636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
#DROP NaN Values
df_final = toronto_merged.dropna().reset_index(drop=True)

# Chage `Cluster Labels` to integer
df_final['Cluster Labels'] = df_final['Cluster Labels'].astype('int')
df_final.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,43.753259,-79.329656,2,Fast Food Restaurant,Park,Food & Drink Shop,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dessert Shop
1,North York,Victoria Village,43.725882,-79.315572,0,Pizza Place,Coffee Shop,Hockey Arena,Portuguese Restaurant,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
2,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Pub,Bakery,Theater,Breakfast Spot,Café,Historic Site,Spa,Beer Store
3,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763,0,Clothing Store,Coffee Shop,Furniture / Home Store,Miscellaneous Shop,Carpet Store,Boutique,Vietnamese Restaurant,Accessories Store,Electronics Store,Eastern European Restaurant
4,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,0,Coffee Shop,Diner,Café,Sushi Restaurant,Yoga Studio,Portuguese Restaurant,Bar,Beer Bar,Spa,Smoothie Shop


In [56]:
df_final.isnull().sum()

Borough                   0
Neighborhood              0
Latitude                  0
Longitude                 0
Cluster Labels            0
1st Most Common Venue     0
2nd Most Common Venue     0
3rd Most Common Venue     0
4th Most Common Venue     0
5th Most Common Venue     0
6th Most Common Venue     0
7th Most Common Venue     0
8th Most Common Venue     0
9th Most Common Venue     0
10th Most Common Venue    0
dtype: int64

In [57]:
# Fisualize clusters

# create map
map_clusters = folium.Map(location=[lat, lon], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_final['Latitude'], df_final['Longitude'], df_final['Neighborhood'], df_final['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine CLusters

In [58]:
# CLuster 0
df_final[df_final['Cluster Labels']==0].iloc[:,[1,5,6,7,8,9,10,11,12,13,14]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Victoria Village,Pizza Place,Coffee Shop,Hockey Arena,Portuguese Restaurant,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
2,"Regent Park , Harbourfront",Coffee Shop,Park,Pub,Bakery,Theater,Breakfast Spot,Café,Historic Site,Spa,Beer Store
3,"Lawrence Manor , Lawrence Heights",Clothing Store,Coffee Shop,Furniture / Home Store,Miscellaneous Shop,Carpet Store,Boutique,Vietnamese Restaurant,Accessories Store,Electronics Store,Eastern European Restaurant
4,Ontario Provincial Government,Coffee Shop,Diner,Café,Sushi Restaurant,Yoga Studio,Portuguese Restaurant,Bar,Beer Bar,Spa,Smoothie Shop
6,Don MillsNorth,Dessert Shop,Gym,Athletics & Sports,Caribbean Restaurant,Japanese Restaurant,Café,Doner Restaurant,Discount Store,Distribution Center,Dog Run
7,"Parkview Hill , Woodbine Gardens",Pizza Place,Pet Store,Athletics & Sports,Gastropub,Intersection,Flea Market,Pharmacy,Breakfast Spot,Bank,Gym / Fitness Center
8,"Garden District, Ryerson",Coffee Shop,Clothing Store,Cosmetics Shop,Japanese Restaurant,Bubble Tea Shop,Café,Italian Restaurant,Middle Eastern Restaurant,Diner,Fast Food Restaurant
9,Glencairn,Bakery,Convenience Store,Japanese Restaurant,Women's Store,Drugstore,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
10,"West Deane Park , Princess Gardens , Martin Gr...",Bakery,Women's Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dim Sum Restaurant
11,"Rouge Hill , Port Union , Highland Creek",Bar,Women's Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dim Sum Restaurant


In [59]:
# CLuster 1
df_final[df_final['Cluster Labels']==1].iloc[:,[1,5,6,7,8,9,10,11,12,13,14]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,Scarborough Village,Playground,Women's Store,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


In [60]:
# CLuster 2
df_final[df_final['Cluster Labels']==2].iloc[:,[1,5,6,7,8,9,10,11,12,13,14]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,Fast Food Restaurant,Park,Food & Drink Shop,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dessert Shop
20,Caledonia-Fairbanks,Park,Pool,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
34,The Danforth East,Intersection,Park,Convenience Store,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
39,DownsviewEast,Park,Airport,Other Repair Shop,Women's Store,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
51,"Willowdale , Newtonbrook",Park,Women's Store,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
63,Weston,Convenience Store,Women's Store,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
65,York Mills West,Flower Shop,Park,Convenience Store,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
84,"Milliken , Agincourt North , Steeles East , L'...",Park,Intersection,Playground,Women's Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
90,Rosedale,Park,Trail,Playground,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop


In [61]:
# CLuster 3
df_final[df_final['Cluster Labels']==3].iloc[:,[1,5,6,7,8,9,10,11,12,13,14]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,"Malvern , Rouge",Fast Food Restaurant,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,College Rec Center


In [62]:
# CLuster 4
df_final[df_final['Cluster Labels']==4].iloc[:,[1,5,6,7,8,9,10,11,12,13,14]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,"York Mills , Silver Hills",Cafeteria,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Women's Store,Farmers Market


# Thank You